In [ ]:
import pandas as pd
import re
from sklearn.utils import shuffle
import random as rand
import numpy as np
import math

In [ ]:
random = 24

In [ ]:
data = pd.read_csv('games.csv')
random = 24
data = data[["Metacritic", "Indie", "Presence", "Platform", 
             "Memory", "RatingsBreakdown", "ReleaseDate", "Soundtrack", 
             "OriginalCost", "DiscountedCost", "Controller",
             "Achievements", "Languages", "Storage", "Tags"]]

In [ ]:
data.describe(include= 'all')

In [ ]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

In [ ]:
actions = {k:[] for k in data.columns}
actions['Indie'].append(lambda x: x == 1.0) 
actions['Controller'].append(lambda x: x == 1.0) 
actions['Soundtrack'].append(lambda x: x == 1.0)
actions['ReleaseDate'].append(lambda x: int(x.split('-')[0]))

def replace_MB(x):
    x = str(x).lower()
    first = re.search(r"\d", x)
    if (first == None):
        return None
    first = first.start()
    i = first+1
    while(i<len(x) and (x[i].isdigit() or x[i] == '.')):
        i = i+1
    last = i
    while(i<len(x) and x[i:i+2] != "gb" and x[i:i+2] != "mb"):
        i = i+1
    if (i == len(x) or x[i:i+2] == "gb"):
        return(float(x[first:last])*1000)
    return(float(x[first:last]))


def clean_originalCost(x):
    x = str(x).replace('$', '')
    try:
        number = float(x)
        return number
    except:
        return 0

def Metacritic_to_Range(x):
    if x > 75:
        return "Very good"
    
    elif x > 50 and x <= 75:
        return "Good"
    
    elif x > 25 and x <= 50:
        return "Bad"
    
    else:
        return "Very bad"
        
    

actions['Metacritic'].append(Metacritic_to_Range)
actions['OriginalCost'].append(lambda x: clean_originalCost(x))
actions['OriginalCost'].append(lambda x: float(str(x).replace('$', '')))
actions['DiscountedCost'].append(lambda x: float(str(x).replace('$', '')))
actions['Storage'].append(lambda x: replace_MB(x))
actions['Memory'].append(lambda x: replace_MB(x))

for name in actions:
    for action in actions[name]:
        data.loc[:,name] = data[data[name].notna()][name].apply(action)

data['DiscountedCost'] = data['DiscountedCost'].fillna(0)
data

In [ ]:
platform = pd.Series(data['Platform']).str.get_dummies(', ')
platform.columns
data['Platform-Mobile'] = platform[['Android','iOS']].sum(axis = 1).apply(lambda x: x > 0) 
data['Platform-PC'] = platform[['Apple II','Atari ST', 'Classic Macintosh', 'Commodore / Amiga',  'Linux', 'PC', 'Web', 'macOS']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-Xbox'] = platform[['Xbox', 'Xbox 360', 'Xbox One']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-PlayStation'] = platform[['PS Vita', 'PSP', 'PlayStation', 'PlayStation 2', 'PlayStation 3', 'PlayStation 4']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-Nintendo'] = platform[['Game Boy', 'GameCube', 'Nintendo 3DS', 'Nintendo DS', 'Nintendo Switch', 'Wii', 'Wii U']].sum(axis = 1).apply(lambda x: x > 0) 
data['Platform-Sega'] = platform[['Dreamcast','Game Gear','Genesis','SEGA Master System', 'SEGA Saturn']].sum(axis = 1).apply(lambda x: x > 0) 
data

In [ ]:
def RatingsBreakdown(x):
    x = x.split(', ')
    result = [np.nan, np.nan, np.nan, np.nan]
    for i in x:
        if i.split(' ')[0] == 'recommended:':
            result[0] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'meh:':
            result[1] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'exceptional:':
            result[2] = float(i.split(' ')[1])
        elif i.split(' ')[0] == 'skip:':
            result[3] = float(i.split(' ')[1])
    return result  
        
df_aux = pd.DataFrame(data[data['RatingsBreakdown'].notna()]['RatingsBreakdown'].apply(lambda x: RatingsBreakdown(x)))
df_aux = pd.DataFrame(df_aux.RatingsBreakdown.values.tolist(),index = data[data['RatingsBreakdown'].notna()]['RatingsBreakdown'].index, columns=['RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip'])
data = data.join(df_aux)
data

In [ ]:
Tags = pd.Series(data['Tags']).str.get_dummies(', ').apply(lambda x: x > 0)
del Tags['Indie']
del Tags['Soundtrack']
del Tags['Controller']
Languages = pd.Series(data['Languages']).str.get_dummies(', ').apply(lambda x: x > 0)


data = data.join(Tags)
data = data.join(Languages)

In [ ]:
del data['RatingsBreakdown']
del data['Platform']
del data['Tags']
del data['Languages']

In [ ]:
for name in data.columns:
    print(name, data[name].isna().sum()/2000)

In [ ]:
rand.seed(random)
data = data.dropna(subset=['Metacritic'], axis='index')
x = [y for y in range(len(data))]
x = rand.sample(x, 2200)
data = data.iloc[x]
data

In [ ]:
for col in data.columns:
    if len(data[col].unique()) == 1:
        data.drop(col,inplace=True,axis=1)

In [ ]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

In [ ]:
from scipy import stats

In [ ]:
col =['Presence', 'Memory', 'ReleaseDate', 'Achievements', 'Storage']
out = None
for name in col:
    data = data[~(data[name] - data[name].mean() > 3*data[name].std())]

In [ ]:
data

In [ ]:
data.to_csv('games_clean.csv', index=False)